In [11]:
import nltk
import re
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import matplotlib.pyplot as plt
import json
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [12]:
with open("data/ProgrammWebScrapy.txt", 'rb') as data:
    df = pd.read_json(data)

In [13]:
#shows first lines
df['Description']

0        Google Maps' services have been split into mul...
1        It has been split into multiple APIs, includin...
2        The Data API allows users to integrate their p...
3        The Flickr API can be used to retrieve photos ...
4        Its functions have been split among the follow...
5        What was formerly the ECSeCommerce Servicehas ...
6        Twilio provides a simple hosted API and markup...
7        The Last.fm API gives users the ability to bui...
8        eBay's primary web services are found in the t...
9        Twilio provides a simple hosted API and markup...
10       Bing Maps API and Interactive SDK features an ...
11       del.icio.us is a social bookmarking website th...
12       Use the Google App Engine platform to run web ...
13       The Foursquare API provides location based exp...
14       The Google Gadgets API provides a way to put t...
15       DocuSign is a Cloud based legally compliant eS...
16       Since 2006 Amazon Web Services has been offeri.

In [14]:
#shows columns
df.columns

Index(['APIEndpoint', 'APIForumMessageBoards', 'APIPortalHomePage',
       'APIProvider', 'ArchitecturalStyle', 'AuthenticationModel',
       'Description', 'DeviceSpecific', 'DocsHomePageUR',
       'IsThisanUnofficialAPI', 'IstheAPIDesign', 'PrimaryCategory',
       'RestrictedAccess', 'SSLSupport', 'Scope', 'SecondaryCategories',
       'SupportEmailAddress', 'SupportedRequestFormats',
       'SupportedResponseFormats', 'Title'],
      dtype='object')

In [15]:
# remove ponctuation 
punctuation_signs = list("?:!.,;({[&#<>]})'1234567890\"")
print(punctuation_signs)
stop_words = list(stopwords.words('english'))###########################
def clean_text():
    # Dataframe creation
    lemmatized_text_list = []
    data = pd.DataFrame(df,columns=['Description','Title','PrimaryCategory','SecondaryCategories'])
   # print(data['Description'])
   # df.loc[0] = text
    data['Content_Parsed_1'] = data['Description'].str.replace("\r", " ")
    data['Content_Parsed_1'] = data['Content_Parsed_1'].str.replace("\n", " ")
    data['Content_Parsed_1'] = data['Content_Parsed_1'].str.replace("    ", " ")
    data['Content_Parsed_1'] = data['Content_Parsed_1'].str.replace('"', '')
    data['Content_Parsed_2'] = data['Content_Parsed_1'].str.lower()
    data['Content_Parsed_3'] = data['Content_Parsed_2']
    print(data['Content_Parsed_3'][157])
    for punct_sign in punctuation_signs:
        data['Content_Parsed_3'] = data['Content_Parsed_3'].str.replace(punct_sign, '')
    print(data['Content_Parsed_3'][157])
    data['Content_Parsed_3'] = data['Content_Parsed_3'].str.replace("'s", "")
    data['Content_Parsed_4'] = data['Content_Parsed_3'].str.replace("s'", "")
    
   #####################################################################
    wordnet_lemmatizer = WordNetLemmatizer()
    #########################
    nrows = len(data)
    lemmatized_text_list = []
    for row in range(0, nrows):
        # Create an empty list containing lemmatized words
        lemmatized_list = []
        # Save the text and its words into an object
        text = data.loc[row]['Content_Parsed_4']
        text_words = text.split(" ")
        # Iterate through every word to lemmatize
        for word in text_words:
            lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        # Join the list
        lemmatized_text = " ".join(lemmatized_list)
        # Append to the list containing the texts
        lemmatized_text_list.append(lemmatized_text)
    data['Content_Parsed_5'] = lemmatized_text_list
    data['Content_Parsed_6'] = data['Content_Parsed_5']
    data['Content_Parsed_6'].replace('', np.nan,inplace=True)
   # df['Tenant'].replace('', np.nan, inplace=True)
    #data['Content_Parsed_6'] = data['Content_Parsed_6'].str.replace("  ", " ")
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        data['Content_Parsed_6'] = data['Content_Parsed_6'].str.replace(regex_stopword,'')
    data['Content_Parsed_6'] =  data['Content_Parsed_6'].str.replace("\s\s+", ' ')
   # data = data['Content_Parsed_6']
    code_cat={}
    data.fillna('unknow', inplace=True)
    #data.dropna(subset=['Content_Parsed_6'])
   # data.loc[350]['data']="no description"
    data.loc[350]
    for i, val in enumerate(data['PrimaryCategory'].unique()):
       # print(i)
        code_cat[str(val)] = i
    data['class_code'] = data['PrimaryCategory']
    data = data.replace({'class_code':code_cat})
    data = data.rename(columns={'Content_Parsed_6': 'data','Title':'name','PrimaryCategory':'class_1','SecondaryCategories':'class_2'})
    dataset=pd.DataFrame(data,columns=['data','name','class_1','class_code','class_2'])
    #return data
##############################################################################
    # TF-IDF
   # print(dataset.head())
    return dataset

    
dataset=clean_text()

['?', ':', '!', '.', ',', ';', '(', '{', '[', '&', '#', '<', '>', ']', '}', ')', "'", '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '"']
its functions have been split among the following apis: world weather online city search, world weather online time zone, world weather online ski and mountain, world weather online historical or past weather, world weather online local city weather, world weather online marine, sailing, surfing weather, and world weather online ski weather. this profile is being maintained purely for historical and research purposes.] world weather online provides weather forecast and weather api content for worldwide city and towns. the developer weather api allows developers and programmers to access free 5 day weather feed in xml, json and csv format for any location by uk postcode, canada postal code, us zipcode, city and town name (country and state/region filter) or by using latitude and longitude.
its functions have been split among the following apis worl

In [16]:
dataset.loc[350]['data']

'unknow'

In [17]:
dataset.loc[350]

data                            unknow
name          Windows Live Gadgets API
class_1                        Widgets
class_code                          11
class_2                         unknow
Name: 350, dtype: object

In [18]:
dataset.to_csv('data/microservices.csv', encoding='utf-8', index=False, header=True)